# Input-Output Analysis and Modelling with MARIO 

##### 12th Edition of the International School of IO Analysis

Authors: Nicolò Golinucci, Lorenzo Rinaldi & Matteo Vincenzo Rocco

Affiliated to eNextGen, Politecnico di Milano

---


## Welcome to the School!
In this module, we are going to show how to use MARIO, an open-source software based on Python, inspired from [pymrio](https://pymrio.readthedocs.io/en/latest/), designed to handle any SUT or IOT table and perform easy calculations and modelling analyses.

In particular, today we are going to navigate through the following functionalities:
- Downloading & Parsing
- Aggregation
- Navigation utilities
- SUT-to-IOT transformation
- Shock analyses


### Downloading & Parsing
MARIO allows for parsing a variety of standardly adopted databases, in particular: 
- EXIOBASE, in many different versions:
    - Monetary SUTs v3.8.2, available on [Zenodo](https://zenodo.org/records/5589597) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_exiobase_monetary/tutorial_parse_exiobase_monetary.html))
    - Monetary IOTs v3.8.2, available on [Zenodo](https://zenodo.org/records/5589597) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_exiobase_monetary/tutorial_parse_exiobase_monetary.html))
    - Hybrid SUT v3.3.18, available on [Zenodo](https://zenodo.org/records/7244919) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_exiobase_hybrid/tutorial_parse_exiobase_hybrid.html))

- EORA, both single- and multi-regional tables (EORA26), available on its [official website](https://www.worldmrio.com) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_eora/tutorial_parse_eora.html))

- FIGARO (only SUT tables), available on [Eurostat](https://ec.europa.eu/eurostat/web/esa-supply-use-input-tables/database) (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_figaro_parser/tutorial_figaro_parser.html))

- Custom databases from Excel files or txt (documented MARIO example [here](https://mario-suite.readthedocs.io/en/latest/examples/tutorial_parse_custom_database/tutorial_parse_custom_database.html))


#### Practical example: FIGARO download & parser

In this example we will see how to parse FIGARO supply-use tables.
First we need to provide the path in your computer where you desire to store the table. Then the 'download_figaro' method will need just the type of table (SUT or IOT), the year and, of course, the path.

In [ ]:
# (Optional but recommended step) MARIO dependencies are being updated, and you might have future warnings related to them. We switch them off!
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import mario

save_path = "database/FIGARO/2021"

# Download Figaro
mario.download_figaro(
    table = "SUT",
    year = 2021,
    path = save_path

)

Once downloaded, use the 'parse_FIGARO_SUT' method, providing the path where you stored the downloaded tables. MARIO will create a Database object, which we call 'world' since FIGARO is a global MRIO table (this choice is of course arbitrary)

In [ ]:
world = mario.parse_FIGARO_SUT(save_path)

#### Aggregation
For the sake of keeping things simple at the beginning, we proceed to aggregate the table into a smaller ones so that we can check all the next steps in parallel with an Excel conterfactual. 
MARIO allows the user to fill an Excel template to help the user provide the information on what and how to aggregate via the "get_aggregation_excel" method

In [ ]:
path_aggr = 'database/FIGARO/Aggregations/conceptual_aggregation_blank.xlsx'

world.get_aggregation_excel(
    path = path_aggr,
)

Once the Excel template is filled, the user can pass the information to MARIO via the "aggregate" method

In [ ]:
path_aggr = 'database/FIGARO/Aggregations/conceptual_aggregation_filled.xlsx'

world.aggregate(
    io=path_aggr,
    levels = [
        "Region",
        "Activity",
        "Commodity"
    ]
)

#### Exporting table to Excel

It is possible to export the table to Excel (or to a set of txt files) adopting the "to_excel" method. By default, MARIO will export the absolute values (or "flows") matrices and not the coefficients (which can actually be requested)

In [ ]:
path_concept_table = f'database/FIGARO/conceptual_excel_table.xlsx'

world.to_excel(
    path=path_concept_table,
    flows=True,
    # coefficients=True, # Uncomment to include coefficients
    )

#### Navigating table's properties

Once parsed, it is possible to check some basic information and properties characterizing the table. Just by calling the name of the database object, MARIO returns the sets of the tables in terms of number of regions, activities, commodities and others.

In [ ]:
world

It is also useful to get a list of a given set, via the "get_index": it is enough to pass the name of the set, which must be with the FIRST CAPITAL LETTER and SINGULAR

In [ ]:
world.get_index('Activity')

In [ ]:
world.get_index('Region')

##### Navigating & Calculating table's matrices

Additionally, it is possible to go through the table's matrices and calculate new ones (i.e. coefficients).
To get a matrix from a database object, it is enough to call the attribute with the name of the matrix itself: MARIO will return the matrix, calculating it first if not available.

N.B. Terminology adopted in MARIO is available in the [documentation](https://mario-suite.readthedocs.io/en/latest/terminology.html)

In [ ]:
world.Y

In [ ]:
world.is_balanced('coefficients')

##### Adding environmental extensions

A table can be environmentally extended with custom satellite accounts in case they are missing.
MARIO allows to do so via the "add_extensions" method. Before that, as for the aggregation, it is possible to call the "get_extensions_excel" to ask MARIO for a template to fill with the desired data. Accounts can be provided in the "V" (think of hybrid-units tables for instance) or in the "E" matrices.

We are going to add the extensions to the full FIGARO table. Therefore we are going to parse it back from where we downloaded at the beginning

In [ ]:
world = mario.parse_FIGARO_SUT('database/FIGARO/2021')

After re-parsing, chose a path where to print the Excel template and call the 'get_extensions_excel' method, specifying also the matrix equal to 'E'

In [ ]:
extensions_path = 'database/FIGARO/Extensions/extensions_blank.xlsx'
world.get_extensions_excel(path=extensions_path, matrix='E')

For this occasion, we already prepared the emissions accounts, taken from the Exiobase database after aggregating it according to FIGARO resolution.
To use the 'add_extensions' method, it is also necessary to prepare a DataFrame with the units (it can be done also preparing it in Excel)

In [ ]:
units = pd.DataFrame('kg',index=['CO2','CH4','N2O'],columns=['units'])
world.add_extensions(io='database/FIGARO/Extensions/extensions_filled.xlsx', matrix='E',units=units)